In [4]:
# Notebook : notebooks/ingestion/01_ingestion_sursaud_regional.ipynb
import pandas as pd
import numpy as np
from pathlib import Path

print("🗺️ INGESTION DES DONNÉES RÉGIONALES SURSAUD")
print("=" * 50)
base_path = Path("/Users/mawensalignat-moandal/Documents/Engineering/school/S3/hackaton/vaccination-predictive-lab/data/raw/sursaud")

🗺️ INGESTION DES DONNÉES RÉGIONALES SURSAUD


In [5]:
# Cell 1: Chargement des données régionales
print("📂 Chargement des données régionales...")

df_regional = pd.read_csv(base_path / "grippe-regionales.csv", sep=",", encoding="utf-8")

print(f"✅ Données régionales chargées : {df_regional.shape}")

📂 Chargement des données régionales...
✅ Données régionales chargées : (27180, 8)


In [6]:
# Cell 2: Analyse de la structure des données
print("\n🔍 ANALYSE DE LA STRUCTURE")
print("=" * 35)

print(f"📊 Volume total : {df_regional.shape[0]:,} observations")
print(f"📅 Période : {df_regional['1er jour de la semaine'].min()} à {df_regional['1er jour de la semaine'].max()}")

# Analyse des régions
regions = df_regional['Région'].nunique()
print(f"🗺️ Régions : {regions}")

# Analyse des classes d'âge
classes_age = df_regional['Classe d\'âge'].nunique()
print(f"👥 Classes d'âge : {classes_age}")

print(f"\n🗺️ Top 10 régions par nombre d'observations :")
region_counts = df_regional['Région'].value_counts().head(10)
for i, (region, count) in enumerate(region_counts.items(), 1):
    print(f"{i:2d}. {region}: {count:,} observations")


🔍 ANALYSE DE LA STRUCTURE
📊 Volume total : 27,180 observations
📅 Période : 2019-12-30 à 2025-10-06
🗺️ Régions : 18
👥 Classes d'âge : 5

🗺️ Top 10 régions par nombre d'observations :
 1. Mayotte: 1,510 observations
 2. Île-de-France: 1,510 observations
 3. Guadeloupe: 1,510 observations
 4. Auvergne et Rhône-Alpes: 1,510 observations
 5. Occitanie: 1,510 observations
 6. Provence-Alpes-Côte d'Azur: 1,510 observations
 7. Nouvelle Aquitaine: 1,510 observations
 8. Bretagne: 1,510 observations
 9. Centre-Val de Loire: 1,510 observations
10. Bourgogne et Franche-Comté: 1,510 observations


In [7]:
# Cell 3: Nettoyage des colonnes
print("\n🧹 Nettoyage des colonnes...")

# Renommage des colonnes pour cohérence
df_regional.columns = [
    'date_debut_semaine', 'semaine', 'code_region', 'region', 'classe_age',
    'taux_urgences_grippe', 'taux_hospitalisations_grippe', 'taux_sos_medecins_grippe'
]

# Conversion des dates
df_regional['date_debut_semaine'] = pd.to_datetime(df_regional['date_debut_semaine'])

print("✅ Colonnes nettoyées")


🧹 Nettoyage des colonnes...
✅ Colonnes nettoyées


In [8]:
# Cell 4: Ajout des composants temporels
print("\n📅 Ajout des composants temporels...")

df_regional['annee'] = df_regional['date_debut_semaine'].dt.year
df_regional['mois'] = df_regional['date_debut_semaine'].dt.month
df_regional['semaine_annee'] = df_regional['date_debut_semaine'].dt.isocalendar().week
df_regional['jour_semaine'] = df_regional['date_debut_semaine'].dt.dayofweek
df_regional['trimestre'] = df_regional['date_debut_semaine'].dt.quarter

print("✅ Composants temporels ajoutés")



📅 Ajout des composants temporels...
✅ Composants temporels ajoutés


In [9]:
# Cell 5: Gestion des valeurs manquantes
print("\n🔍 Gestion des valeurs manquantes...")

# Analyse des valeurs manquantes
missing_data = df_regional.isnull().sum()
print("Valeurs manquantes par colonne :")
for col, missing in missing_data.items():
    if missing > 0:
        print(f"   ⚠️ {col}: {missing:,} ({missing/len(df_regional)*100:.1f}%)")
    else:
        print(f"   ✅ {col}: Aucune valeur manquante")

# Remplacement des valeurs manquantes par 0 (logique pour les taux)
df_regional['taux_sos_medecins_grippe'] = df_regional['taux_sos_medecins_grippe'].fillna(0)

print("✅ Valeurs manquantes traitées")


🔍 Gestion des valeurs manquantes...
Valeurs manquantes par colonne :
   ✅ date_debut_semaine: Aucune valeur manquante
   ✅ semaine: Aucune valeur manquante
   ✅ code_region: Aucune valeur manquante
   ✅ region: Aucune valeur manquante
   ✅ classe_age: Aucune valeur manquante
   ⚠️ taux_urgences_grippe: 890 (3.3%)
   ⚠️ taux_hospitalisations_grippe: 897 (3.3%)
   ⚠️ taux_sos_medecins_grippe: 6,045 (22.2%)
   ✅ annee: Aucune valeur manquante
   ✅ mois: Aucune valeur manquante
   ✅ semaine_annee: Aucune valeur manquante
   ✅ jour_semaine: Aucune valeur manquante
   ✅ trimestre: Aucune valeur manquante
✅ Valeurs manquantes traitées


In [10]:
# Cell 6: Validation de la qualité des données
print("\n📊 VALIDATION DE LA QUALITÉ")
print("=" * 35)

# Vérification des codes région
print(f"🗺️ Codes région uniques : {df_regional['code_region'].nunique()}")
print(f"🗺️ Noms région uniques : {df_regional['region'].nunique()}")

# Vérification des classes d'âge
print(f"👥 Classes d'âge : {sorted(df_regional['classe_age'].unique())}")

# Vérification des valeurs négatives (anormales pour des taux)
negative_urgences = (df_regional['taux_urgences_grippe'] < 0).sum()
negative_hosp = (df_regional['taux_hospitalisations_grippe'] < 0).sum()

if negative_urgences > 0:
    print(f"⚠️ Valeurs négatives dans taux_urgences_grippe : {negative_urgences}")
else:
    print(f"✅ Aucune valeur négative dans taux_urgences_grippe")

if negative_hosp > 0:
    print(f"⚠️ Valeurs négatives dans taux_hospitalisations_grippe : {negative_hosp}")
else:
    print(f"✅ Aucune valeur négative dans taux_hospitalisations_grippe")



📊 VALIDATION DE LA QUALITÉ
🗺️ Codes région uniques : 18
🗺️ Noms région uniques : 18
👥 Classes d'âge : ['00-04 ans', '05-14 ans', '15-64 ans', '65 ans ou plus', 'Tous âges']
✅ Aucune valeur négative dans taux_urgences_grippe
✅ Aucune valeur négative dans taux_hospitalisations_grippe


In [ ]:
# Cell 7: Statistiques par région
print("\n STATISTIQUES PAR RÉGION")
print("=" * 35)

# Statistiques par région
region_stats = df_regional.groupby('region').agg({
    'taux_urgences_grippe': ['count', 'mean', 'std', 'max'],
    'taux_hospitalisations_grippe': ['mean', 'std', 'max'],
    'taux_sos_medecins_grippe': ['mean', 'std', 'max']
}).round(2)

print("📊 Statistiques par région :")
for region in df_regional['region'].unique():
    region_data = df_regional[df_regional['region'] == region]
    count = len(region_data)
    avg_urgences = region_data['taux_urgences_grippe'].mean()
    print(f"   {region}: {count:,} observations, {avg_urgences:.1f} urgences moy.")


📊 STATISTIQUES PAR RÉGION
📊 Statistiques par région :
   Mayotte: 1,510 observations, 375.0 urgences moy.
   Île-de-France: 1,510 observations, 870.4 urgences moy.
   Hauts-de-France: 1,510 observations, 710.7 urgences moy.
   Normandie: 1,510 observations, 620.0 urgences moy.
   Pays de la Loire: 1,510 observations, 581.8 urgences moy.
   Martinique: 1,510 observations, 480.9 urgences moy.
   Guyane: 1,510 observations, 1053.9 urgences moy.
   Réunion: 1,510 observations, 685.4 urgences moy.
   Grand Est: 1,510 observations, 697.1 urgences moy.
   Bourgogne et Franche-Comté: 1,510 observations, 700.2 urgences moy.
   Centre-Val de Loire: 1,510 observations, 630.0 urgences moy.
   Bretagne: 1,510 observations, 499.6 urgences moy.
   Nouvelle Aquitaine: 1,510 observations, 644.2 urgences moy.
   Provence-Alpes-Côte d'Azur: 1,510 observations, 916.7 urgences moy.
   Occitanie: 1,510 observations, 562.6 urgences moy.
   Auvergne et Rhône-Alpes: 1,510 observations, 694.3 urgences moy.
   

In [14]:
# Cell 8: Sauvegarde des données nettoyées
print("\n Sauvegarde...")

df_regional.to_csv("../../data/cleaned/sursaud_regional_cleaned.csv", index=False)

print("✅ Données régionales sauvegardées : sursaud_regional_cleaned.csv")




 Sauvegarde...
✅ Données régionales sauvegardées : sursaud_regional_cleaned.csv


In [15]:
# Cell 9: Aperçu des données nettoyées
print("\n🔍 APERÇU DES DONNÉES NETTOYÉES")
print("=" * 40)

print("📊 Shape final :", df_regional.shape)
print("📅 Période :", df_regional['date_debut_semaine'].min(), "à", df_regional['date_debut_semaine'].max())

print("\n📊 Aperçu des données :")
display(df_regional.head())

print("\n📊 Statistiques descriptives :")
display(df_regional.describe())


🔍 APERÇU DES DONNÉES NETTOYÉES
📊 Shape final : (27180, 13)
📅 Période : 2019-12-30 00:00:00 à 2025-10-06 00:00:00

📊 Aperçu des données :


,date_debut_semaine,semaine,code_region,region,classe_age,taux_urgences_grippe,taux_hospitalisations_grippe,taux_sos_medecins_grippe,annee,mois,semaine_annee,jour_semaine,trimestre
0,2023-02-20,2023-S08,6,Mayotte,00-04 ans,383.141762,1562.500000,0.0,2023,2,8,0,1
1,2023-02-20,2023-S08,6,Mayotte,15-64 ans,728.597450,0.000000,0.0,2023,2,8,0,1
2,2023-02-20,2023-S08,6,Mayotte,65 ans ou plus,2127.659574,0.000000,0.0,2023,2,8,0,1
3,2023-02-27,2023-S09,6,Mayotte,05-14 ans,714.285714,0.000000,0.0,2023,2,9,0,1
4,2023-02-27,2023-S09,6,Mayotte,65 ans ou plus,4166.666667,14285.714286,0.0,2023,2,9,0,1



📊 Statistiques descriptives :


,date_debut_semaine,code_region,taux_urgences_grippe,taux_hospitalisations_grippe,taux_sos_medecins_grippe,annee,mois,semaine_annee,jour_semaine,trimestre
count,27180,27180.000000,26290.000000,26283.000000,27180.000000,27180.000000,27180.000000,27180.0,27180.0,27180.000000
mean,2022-11-17 12:00:00.000000256,39.388889,681.709809,518.062581,2728.560887,2022.394040,6.341060,25.84106,0.0,2.447020
min,2019-12-30 00:00:00,1.000000,0.000000,0.000000,0.000000,2019.000000,1.000000,1.0,0.0,1.000000
25%,2021-06-07 00:00:00,6.000000,23.984064,0.000000,0.000000,2021.000000,3.000000,13.0,0.0,1.000000
50%,2022-11-17 12:00:00,30.000000,119.608809,0.000000,566.572238,2022.000000,6.000000,26.0,0.0,2.000000
75%,2024-04-29 00:00:00,75.000000,578.073841,285.578361,2731.646146,2024.000000,9.000000,38.0,0.0,3.000000
max,2025-10-06 00:00:00,94.000000,22580.645161,33333.333333,60975.609756,2025.000000,12.000000,53.0,0.0,4.000000
std,NaN,32.287741,1502.111519,1471.596994,5207.647104,1.673855,3.390676,14.796501,0.0,1.098864


In [17]:
# Cell 10: Comparaison avec les données départementales
print("\n🔍 COMPARAISON AVEC LES DONNÉES DÉPARTEMENTALES")
print("=" * 55)

# Chargement des données départementales pour comparaison
df_departement = pd.read_csv("../../data/cleaned/sursaud_departement_cleaned.csv")
df_departement['date_debut_semaine'] = pd.to_datetime(df_departement['date_debut_semaine'])

print(f"📊 Données régionales : {df_regional.shape}")
print(f"📊 Données départementales : {df_departement.shape}")

print(f"\n📅 Périodes :")
print(f"   Régional : {df_regional['date_debut_semaine'].min()} à {df_regional['date_debut_semaine'].max()}")
print(f"   Département : {df_departement['date_debut_semaine'].min()} à {df_departement['date_debut_semaine'].max()}")

print(f"\n🗺️ Régions :")
print(f"   Régional : {df_regional['region'].nunique()}")
print(f"   Département : {df_departement['region'].nunique()}")

# Vérification de la cohérence
regions_regional = set(df_regional['region'].unique())
regions_departement = set(df_departement['region'].unique())

if regions_regional == regions_departement:
    print(f"\n✅ Cohérence parfaite : Mêmes régions dans les deux datasets")
else:
    print(f"\n⚠️ Différences régionales :")
    print(f"   Régional uniquement : {regions_regional - regions_departement}")
    print(f"   Département uniquement : {regions_departement - regions_regional}")



🔍 COMPARAISON AVEC LES DONNÉES DÉPARTEMENTALES
📊 Données régionales : (27180, 13)
📊 Données départementales : (157040, 15)

📅 Périodes :
   Régional : 2019-12-30 00:00:00 à 2025-10-06 00:00:00
   Département : 2019-12-30 00:00:00 à 2025-10-06 00:00:00

🗺️ Régions :
   Régional : 18
   Département : 21

⚠️ Différences régionales :
   Régional uniquement : set()
   Département uniquement : {'Saint-Barthélemy', 'Saint-Martin', 'Miquelon-Langlade et Saint Pierre'}


In [18]:
# Cell 11: Rapport de synthèse
print("\n📋 RAPPORT DE SYNTHÈSE RÉGIONAL")
print("=" * 45)

total_obs = len(df_regional)
total_regions = df_regional['region'].nunique()
total_classes = df_regional['classe_age'].nunique()

print(f"✅ DONNÉES RÉGIONALES TRAITÉES")
print(f"   📊 Observations totales : {total_obs:,}")
print(f"   🗺️ Régions : {total_regions}")
print(f"   👥 Classes d'âge : {total_classes}")

print(f"\n🎯 AVANTAGES DES DONNÉES RÉGIONALES")
print(f"   📊 Granularité intermédiaire : Entre national et départemental")
print(f"   🔗 Fusion directe : Avec IQVIA régional")
print(f"   📈 Volume suffisant : {total_obs:,} observations")
print(f"   🗺️ Couverture complète : Toutes les régions françaises")

print(f"\n🚀 PROCHAINES ÉTAPES")
print(f"   🔗 Fusion avec IQVIA régional")
print(f"   ⚙️ Feature Engineering régional")
print(f"   🤖 Modèle prédictif régional")
print(f"   📊 Dashboard régional")

print(f"\n🎉 INGESTION RÉGIONALE TERMINÉE !")


📋 RAPPORT DE SYNTHÈSE RÉGIONAL
✅ DONNÉES RÉGIONALES TRAITÉES
   📊 Observations totales : 27,180
   🗺️ Régions : 18
   👥 Classes d'âge : 5

🎯 AVANTAGES DES DONNÉES RÉGIONALES
   📊 Granularité intermédiaire : Entre national et départemental
   🔗 Fusion directe : Avec IQVIA régional
   📈 Volume suffisant : 27,180 observations
   🗺️ Couverture complète : Toutes les régions françaises

🚀 PROCHAINES ÉTAPES
   🔗 Fusion avec IQVIA régional
   ⚙️ Feature Engineering régional
   🤖 Modèle prédictif régional
   📊 Dashboard régional

🎉 INGESTION RÉGIONALE TERMINÉE !
